In [1]:
from xml.etree import ElementTree

anno_file_path = "E:/MVI_39031_v3.xml"
tree = ElementTree.parse(anno_file_path)
root = tree.getroot()
root.attrib

{'name': 'MVI_39031'}

In [10]:
frame =  next(iter(root.findall("./frame")))
targets = frame.findall(".//target")
target = next(iter(targets))
box = target.find("occlusion")


True

In [11]:
import json

with open('E:/anno_pub_detection.json', 'rt') as in_file:
    content = json.load(in_file)

In [13]:
content.keys()

dict_keys(['metadata', 'samples'])

In [19]:
sample = content['samples']['MOT17-02-DPM']
sample.keys()

dict_keys(['entities', 'metadata'])

In [20]:
sample['metadata']

{'data_path': 'train/MOT17-02-DPM/img1',
 'fps': 30.0,
 'number_of_frames': 600,
 'resolution': {'height': 1080, 'width': 1920}}